In [ ]:
%load_ext autoreload
%autoreload 2
from RTKToArrayConversion import *
from ExtendedConeBeamDCC import *
from ConeBeamDCCWithBackprojectionPlane import *
from AllAcquisitionClass import *

In [ ]:
filesdir_ref = ""
# reading projections
proj = itk.imread(filesdir_ref+"sub_corrected_proj.mha")
# Reading the geometry of the scanner
xmlreader = rtk.ThreeDCircularProjectionGeometryXMLFileReader.New()
xmlreader.SetFilename(filesdir_ref+'sub_geometry.xml')
# xmlreader.SetFilename(filesdir_ref+"geometry.xml")
xmlreader.GenerateOutputInformation()
geometry = xmlreader.GetOutputObject()
print('nproj = %d'%(len(geometry.GetGantryAngles())))
print(proj.GetLargestPossibleRegion().GetSize())

In [ ]:
geometry_array = RTKtoNP(geometry)
proj_array = itk.GetArrayFromImage(proj)
proj_infos = GetProjectionInformations(proj)
source_pos_array = GetSourcePositions(geometry)
rotation_matrices_array = GetRotationMatrices(geometry)
fixed_matrices_array = GetFixedSystemMatrices(geometry)
print(proj_infos)

AcquiDCC = DCCWithBPinAnAcquisition(geometry_array, source_pos_array, rotation_matrices_array, fixed_matrices_array, proj_array, proj_infos)
AcquiDCC.ComputeAllPossiblePairs()

In [ ]:
# Fonction non parallélisée 
# AcquiDCC.ComputeDCCForAllPairs()

AcquiDCC.ComputeDCCForAllPairsPara()
#AcquiDCC.pairs

In [ ]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(range(AcquiDCC.n_proj))
G.add_weighted_edges_from(AcquiDCC.pairs)
print(G.number_of_nodes(), G.number_of_edges())
length, path = nx.single_source_dijkstra(G, 0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(geometry_array[8, :],[length[i] for i in range(AcquiDCC.n_proj)])
plt.show()